In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import datetime
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input, concatenate, K, Reshape, LSTM, CuDNNLSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
from keras.losses import mean_squared_error
from sklearn.preprocessing import LabelEncoder, minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import xml.etree.ElementTree as ET
import sklearn
import itertools
import cv2
import scipy
import os
import csv
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
%matplotlib inline
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['BITS AIC 2019 - Reflexis Raw Dataset.csv', 'Store Attributes - Store Attributes.csv']


In [83]:
#Objective : Calculate sales/hour for a given system of system and manager scheduled hours
""""The solution should also identify the impact of any
changes made by the manager on the system
generated schedule and determine which
managers make better changes to the system
generated schedule or more generally, what
factors result in better schedule adjustments by
managers vs changes that result in worse / non-
optimal labor schedule & costs.
Objectives : Maximise sales hour
"""

'"The solution should also identify the impact of any\nchanges made by the manager on the system\ngenerated schedule and determine which\nmanagers make better changes to the system\ngenerated schedule or more generally, what\nfactors result in better schedule adjustments by\nmanagers vs changes that result in worse / non-\noptimal labor schedule & costs.\nObjectives : Maximise sales hour\n'

In [84]:
df = pd.read_csv('../input/BITS AIC 2019 - Reflexis Raw Dataset.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.set_index('DATE')
df.head()

,STORE,TRAFFIC_ACTUAL,TRANSACTIONS_ACTUAL,SALES_ACTUAL,MANAGER_SCHED_HOURS,SYSTEM_SCHED_HOURS
DATE,,,,,,
2016-01-31,203,180,37,2243.07,26.00,20.50
2016-02-01,203,49,22,424.50,28.50,29.75
2016-02-02,203,85,20,878.20,31.50,25.50
2016-02-03,203,84,25,937.80,25.50,27.50
2016-02-04,203,69,13,572.70,25.75,27.25


In [85]:
df['SALES_NORMALIZED'] = minmax_scale(df['SALES_ACTUAL'])
print('Max sales = ', df['SALES_ACTUAL'].max())
print('Number of records =', len(df))
df.head()

Max sales =  113805.47
Number of records = 121062


,STORE,TRAFFIC_ACTUAL,TRANSACTIONS_ACTUAL,SALES_ACTUAL,MANAGER_SCHED_HOURS,SYSTEM_SCHED_HOURS,SALES_NORMALIZED
DATE,,,,,,,
2016-01-31,203,180,37,2243.07,26.00,20.50,0.045756
2016-02-01,203,49,22,424.50,28.50,29.75,0.030201
2016-02-02,203,85,20,878.20,31.50,25.50,0.034081
2016-02-03,203,84,25,937.80,25.50,27.50,0.034591
2016-02-04,203,69,13,572.70,25.75,27.25,0.031468


In [98]:
class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def f1(data, epochs):
    #predict sales given date, store number, system scheduled hours and manager scheduled hours
    batch_size = 128
    split_date = '2018-01-01'
    X_train, Y_train, X_test, Y_test = data[data.index < split_date][['STORE','MANAGER_SCHED_HOURS','SYSTEM_SCHED_HOURS']], data[data.index < split_date]['SALES_ACTUAL'], data[data.index > split_date][['STORE','MANAGER_SCHED_HOURS','SYSTEM_SCHED_HOURS']], data[data.index > split_date]['SALES_ACTUAL']
    
    #X_train, X_test, Y_train, Y_test = train_test_split(data[['STORE','MANAGER_SCHED_HOURS','SYSTEM_SCHED_HOURS']],data['SALES_ACTUAL'], shuffle=False)
    X_train = X_train.values.reshape(X_train.shape[0],1,X_train.shape[1])
    X_test = X_test.values.reshape(X_test.shape[0],1,X_test.shape[1])
    input_shape = (1,3)
    num_classes = 1
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.25, recurrent_dropout=0.25, input_shape=input_shape)))
    model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(Bidirectional(LSTM(64, dropout=0.25, recurrent_dropout=0.25)))
    model.add(Dense(num_classes, activation='relu'))
    model.compile(loss=mean_squared_error, optimizer = Adam(lr = 0.1), metrics=['accuracy'])
    history = model.fit(X_train, Y_train.values, epochs = epochs, steps_per_epoch=int(len(X_train) / batch_size), validation_steps= int(len(X_test)/batch_size), validation_data=(X_test, Y_test.values))
    score = model.evaluate(X_test, Y_test.values)
    return model
    
#def f2(data):
    #maximize sales, by adjusting sys_hrs and mgr_hrs for f1, output (sys_hrs,mgr_hrs)


In [99]:
lstm_model = f1(df, 30)

Train on 112114 samples, validate on 8785 samples
Epoch 1/30
875/875 [==============================] - 184s 211ms/step - loss: 40079213.9246 - acc: 2.2599e-05 - val_loss: 12576942.0000 - val_acc: 0.0000e+00
Epoch 2/30
875/875 [==============================] - 148s 169ms/step - loss: 35641353.0514 - acc: 1.0296e-06 - val_loss: 13727401.0000 - val_acc: 0.0000e+00
Epoch 3/30
875/875 [==============================] - 148s 169ms/step - loss: 35629661.7646 - acc: 0.0000e+00 - val_loss: 13732554.0000 - val_acc: 0.0000e+00
Epoch 4/30
875/875 [==============================] - 148s 169ms/step - loss: 35629663.5109 - acc: 0.0000e+00 - val_loss: 13732573.0000 - val_acc: 0.0000e+00
Epoch 5/30
875/875 [==============================] - 147s 168ms/step - loss: 35629662.7886 - acc: 0.0000e+00 - val_loss: 13732582.0000 - val_acc: 0.0000e+00
Epoch 6/30
875/875 [==============================] - 147s 168ms/step - loss: 35629661.5497 - acc: 0.0000e+00 - val_loss: 13732588.0000 - val_acc: 0.0000e+00
Ep

KeyboardInterrupt: 